Comparison between my implementation of Logistic Regression vs Sklearn's
I'm just a newbie highschooler trying to learn to 'learn' anyway. I really appreciate any comment. SO please comment if you have any suggestion.

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.linear_model import LogisticRegression as LR
e = math.e

In [2]:
train = pd.read_csv('student-mat.csv')
train_y = train[['romantic']]
train = train[['sex', 'age', 'famsize', 'traveltime', 'studytime', 'activities', 'goout', 'famrel']]

In [3]:
train_y['romantic'][train_y['romantic']=='no'] = 0
train_y['romantic'][train_y['romantic']=='yes'] = 1
train['sex'][train['sex']=='F'] = 0
train['sex'][train['sex']=='M'] = 1
train['famsize'][train['famsize']=='LE3'] = 0
train['famsize'][train['famsize']=='GT3'] = 1
train['activities'][train['activities']=='no'] = 0
train['activities'][train['activities']=='yes'] = 1
train = train.apply(pd.to_numeric)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
print(train.info())
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 8 columns):
sex           395 non-null int64
age           395 non-null int64
famsize       395 non-null int64
traveltime    395 non-null int64
studytime     395 non-null int64
activities    395 non-null int64
goout         395 non-null int64
famrel        395 non-null int64
dtypes: int64(8)
memory usage: 24.8 KB
None


,sex,age,famsize,traveltime,studytime,activities,goout,famrel
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,0.473418,16.696203,0.711392,1.448101,2.035443,0.508861,3.108861,3.944304
std,0.499926,1.276043,0.453690,0.697505,0.839240,0.500555,1.113278,0.896659
min,0.000000,15.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,0.000000,16.000000,0.000000,1.000000,1.000000,0.000000,2.000000,4.000000
50%,0.000000,17.000000,1.000000,1.000000,2.000000,1.000000,3.000000,4.000000
75%,1.000000,18.000000,1.000000,2.000000,2.000000,1.000000,4.000000,5.000000
max,1.000000,22.000000,1.000000,4.000000,4.000000,1.000000,5.000000,5.000000


In [5]:
test_ratio = 0.2
train_x = train.values
train_y = train_y.apply(pd.to_numeric).values
sep = int(test_ratio*train.shape[0])
test_x = train_x[:sep]
test_y = train_y[:sep]
train_x = train_x[sep:]
train_y = train_y[sep:]
train_x = np.insert(train_x, 0, np.ones(train_x.shape[0]), 1)
test_x = np.insert(test_x, 0, np.ones(test_x.shape[0]), 1)

In [6]:
np.set_printoptions(threshold=np.nan)
m, n = train_x.shape
def sigmoid(x):
    return 1.0/(1.0+e**(-1.0*x))
def GradientDescent(x, y, theta, learning_rate, numIterations):
    xT = x.transpose()
    theta = theta.reshape(n, 1)
    y = y.reshape(m, 1)
    interval = int(numIterations / 10)
    for i in range(numIterations):
        H = x.dot(theta)
        H = sigmoid(H)
        Loss = H - y
        Gradient = xT.dot(H - y) / m
        theta -= learning_rate * Gradient
        if ((i+1) % interval == 0)or(i==0):
            #Cost = 1/(2*m)*np.sum(Loss**2)
            Cost = -1/m*np.sum((1-y)*np.log(1-H)+y*np.log(H))
            print(i+1, "->", Cost)
    return theta
def DecisionBoundary(x, y):
    np.random.seed(1)
    theta = np.random.random(n)
    learning_rate = 0.005
    numIterations = 100000
    theta = GradientDescent(x, y, theta, learning_rate, numIterations)
    return theta
def LogisticRegression(theta, x):
    x = x.reshape(1, x.shape[-1])
    s = sigmoid(x.dot(theta))
    return float(s[0])
def Score(X, y, theta):
    count = 0
    for i, x in enumerate(X):
        out = 1 if LogisticRegression(theta, x) > 0.5 else 0
        if (out-y[i]==0):
            count += 1
    return count/len(X)
model = DecisionBoundary(train_x, train_y)
Score(test_x, test_y, model)

1 -> 2.65113171292
10000 -> 0.648707447768
20000 -> 0.647606475152
30000 -> 0.647111481688
40000 -> 0.646673260573
50000 -> 0.646276292154
60000 -> 0.645916517697
70000 -> 0.645590467281
80000 -> 0.645294993457
90000 -> 0.645027238356
100000 -> 0.644784607963


0.8227848101265823

In [7]:
sex, age, famsize, traveltime, studytime, activities, goout, famrel = 1, 17, 1, 3, 1, 1, , 5
input_data = np.array([1, sex, age, famsize, traveltime, studytime, activities, goout, famrel])

In [8]:
out = LogisticRegression(model, input_data)
out*100

29.469007101775773

In [9]:
logistic = LR()
logistic.fit(train_x, train_y)
print(logistic.score(test_x, test_y))
float(logistic.predict_proba(input_data.reshape(1, -1))[0][1])*100

0.822784810127


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


30.585727795562544

In [10]:
logistic.coef_

array([[-0.61831704, -0.4647594 ,  0.11811884, -0.29743417,  0.00161876,
        -0.01816651,  0.11768119, -0.09074598, -0.15154086]])

In [11]:
model

array([[ -1.01537746e+00],
       [ -5.02872772e-01],
       [  1.09608408e-01],
       [ -3.27449820e-01],
       [ -7.55581309e-04],
       [ -2.87423670e-02],
       [  1.26371504e-01],
       [ -9.18777141e-02],
       [ -1.54926192e-01]])